In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# <h1 style='background:#C2C4E2; border:0; color:black'><center>DROWSINESS DETECTION</center></h1> 

   <a id='top'></a>
<div class="list-group" id="list-tab" role="tablist">
<h1 style='background:#C2C4E2; border:0; color:black'><center>TABLE OF CONTENTS</center></h1>

1. IMPORTING LIBRARIES
    
2. LOADING DATA   

3. DATA VISUALIZATION AND CLEANINGS     

4. DATA PREPROCESSING     

5. MODEL BUILDING

6. CONCLUSION 

7. END 

 <a id="1"></a>
<h1 style='background:#C2C4E2; border:0; color:black'><center>IMPORTING LIBRARIES</center></h1>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Lambda, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


import os
import cv2

%matplotlib inline
print("Tensorflow version : ", tf.__version__)

 <a id="2"></a>
<h1 style='background:#C2C4E2; border:0; color:black'><center>LOADING DATA</center></h1>

## CHECK LABELS

In [ ]:
labels = os.listdir("../input/drowsiness-dataset/train")
labels

## VISUALIZE SOME IMAGES

In [ ]:
# Visualize a closed eye from the dataset
plt.imshow(plt.imread("../input/drowsiness-dataset/train/Closed/_0.jpg"))

In [ ]:
# Visualize an open eye from the given dataset
plt.imshow(plt.imread("../input/drowsiness-dataset/train/Open/_0.jpg"))

## Get the image array

In [ ]:
a = plt.imread("../input/drowsiness-dataset/train/yawn/10.jpg")

In [ ]:
# Check a.shape
a.shape

## Visualize yawn image

In [ ]:
plt.imshow(plt.imread("../input/drowsiness-dataset/train/yawn/10.jpg"))

Here, we do not need the background, as it is unnecessary.
We only need the face image array.

## For 'yawn' and 'not_yawn', consider only the face (and not the background)

In [ ]:
def face_for_yawn(direc="../input/drowsiness-dataset/train", face_cas_path="../input/prediction-images/haarcascade_frontalface_default.xml"):
    yaw_no = []
    IMG_SIZE = 145
    categories = ["yawn", "no_yawn"]
    for category in categories:
        path_link = os.path.join(direc, category)
        class_num1 = categories.index(category)
        print(class_num1)
        for image in os.listdir(path_link):
            image_array = cv2.imread(os.path.join(path_link, image), cv2.IMREAD_COLOR)
            face_cascade = cv2.CascadeClassifier(face_cas_path)
            faces = face_cascade.detectMultiScale(image_array, 1.3, 5)
            for (x, y, w, h) in faces:
                img = cv2.rectangle(image_array, (x, y), (x+w, y+h), (0, 255, 0), 2)
                roi_color = img[y:y+h, x:x+w]
                resized_array = cv2.resize(roi_color, (IMG_SIZE, IMG_SIZE))
                yaw_no.append([resized_array, class_num1])
    return yaw_no


yawn_no_yawn = face_for_yawn()

## For 'Open' and 'Closed' eyes

In [ ]:
def get_data(dir_path="../input/drowsiness-dataset/train/", face_cas="../input/prediction-images/haarcascade_frontalface_default.xml", eye_cas="../input/prediction-images/haarcascade.xml"):
    labels = ['Closed', 'Open']
    IMG_SIZE = 145
    data = []
    for label in labels:
        path = os.path.join(dir_path, label)
        class_num = labels.index(label)
        class_num +=2
        print(class_num)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
                resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                data.append([resized_array, class_num])
            except Exception as e:
                print(e)
    return data

In [ ]:
data_train = get_data()

## Extend data and convert array

In [ ]:
def append_data():
    yaw_no = face_for_yawn()
    data = get_data()
    yaw_no.extend(data)
    return np.array(yaw_no)

## Store data in a new variable

In [ ]:
new_data = append_data()

In [ ]:
# Seperate labels and features
X = []
y = []
for feature, label in new_data:
    X.append(feature)
    y.append(label)

## Reshape the array

In [ ]:
X = np.array(X)
X = X.reshape(-1,145,145,3)

## Label Binarizer

In [ ]:
label_bin = LabelBinarizer()
y = label_bin.fit_transform(y)

## Label array

In [ ]:
y = np.array(y)

## Split the data into training and testing data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

## Data Augmentation

In [ ]:
train_generator = ImageDataGenerator(rescale = 1/255, zoom_range = 0.2, horizontal_flip = True, rotation_range = 30)
test_generator = ImageDataGenerator(rescale = 1/255)

train_generator = train_generator.flow(np.array(X_train), y_train, shuffle = False)
test_generator = test_generator.flow(np.array(X_test), y_test, shuffle = False)

 <a id="3"></a>
<h1 style='background:#C2C4E2; border:0; color:black'><center>BUILDING MODEL</center></h1>

## Build the model

In [ ]:
model = Sequential([
    Conv2D(256, (3,3), activation = 'relu', input_shape = X_train.shape[1:]),
    MaxPooling2D(pool_size = (2,2)),
    Conv2D(128, (3,3), activation = 'relu'),
    MaxPooling2D(pool_size = (2,2)),
    Conv2D(64, (3,3), activation = 'relu'),
    MaxPooling2D(pool_size = (2,2)),
    Conv2D(32, (3,3), activation = 'relu'),
    MaxPooling2D(pool_size = (2,2)),
    Flatten(),
    Dropout(0.5),
    Dense(64, activation = 'relu'),
    Dense(4, activation = 'softmax')
])

## Compile the model

In [ ]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

## Check the summary of the model

In [ ]:
model.summary()

## Fit the model

In [ ]:
history = model.fit(train_generator, epochs = 50, validation_data = test_generator, shuffle = True, validation_steps = len(test_generator))

 <a id="4"></a>
<h1 style='background:#C2C4E2; border:0; color:black'><center>EVALUATE THE MODEL</center></h1>

## Check the history

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))

plt.plot(epochs, accuracy, "b", label = 'training accuracy')
plt.plot(epochs, val_accuracy, 'r', label = 'validation accuracy')
plt.legend()
plt.show()

plt.plot(epochs, loss, 'b', label = 'training loss')
plt.plot(epochs, val_loss, 'r', label = 'validation loss')
plt.legend()
plt.show()

## Save the model

In [ ]:
model.save("drowiness_new6.h5")

## Make predictions

In [ ]:
prediction = model.predict(X_test) 
prediction = np.argmax(prediction,axis=1)

In [ ]:
# Check the predictions
prediction

## Classification report

In [ ]:
labels_new = ["yawn", "no_yawn", "Closed", "Open"]

In [ ]:
print(classification_report(np.argmax(y_test, axis = 1), prediction, target_names = labels_new))

 <a id="5"></a>
<h1 style='background:#C2C4E2; border:0; color:black'><center>PREDICTION FUNCTION</center></h1>

In [ ]:
labels_new = ["yawn", "no_yawn", "Closed", "Open"]
IMG_SIZE = 145
def prepare(filepath, face_cas="../input/prediction-images/haarcascade_frontalface_default.xml"):
    img_array = cv2.imread(filepath, cv2.IMREAD_COLOR)
    img_array = img_array / 255
    resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return resized_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

model = tf.keras.models.load_model("./drowiness_new6.h5")

- 0 : YAWN
- 1 : NO YAWN
- 2 : CLOSED
- 3 : OPEN

In [ ]:
# prepare("../input/drowsiness-dataset/train/no_yawn/1068.jpg")
prediction = model.predict([prepare("../input/drowsiness-dataset/train/no_yawn/1067.jpg")])
np.argmax(prediction)

In [ ]:
# prepare("../input/drowsiness-dataset/train/no_yawn/1068.jpg")
prediction = model.predict([prepare("../input/drowsiness-dataset/train/yawn/13.jpg")])
np.argmax(prediction)

In [ ]:
# prepare("../input/drowsiness-dataset/train/no_yawn/1068.jpg")
prediction = model.predict([prepare("../input/drowsiness-dataset/train/Closed/_101.jpg")])
np.argmax(prediction)

In [ ]:
# prepare("../input/drowsiness-dataset/train/no_yawn/1068.jpg")
prediction = model.predict([prepare("../input/drowsiness-dataset/train/Open/_104.jpg")])
np.argmax(prediction)

 <a id="6"></a>
<h1 style='background:#C2C4E2; border:0; color:black'><center>FIN</center></h1>